In [1]:
import pandas as pd

In [3]:
data = pd.read_csv("./data/UK_foods.csv")

In [4]:
data.head()

,Unnamed: 0,England,Wales,Scotland,N.Ireland
0,Cheese,105,103,103,66
1,Carcass_meat,245,227,242,267
2,Other_meat,685,803,750,586
3,Fish,147,160,122,93
4,Fats_and_oils,193,235,184,209
